In [ ]:
import boto3
import os


bedrock_client = boto3.client(
    service_name="bedrock",
    region_name="us-west-2"
)


response = bedrock_client.list_inference_profiles()
for profile in response['inferenceProfileSummaries']:
    if "llama" in profile['inferenceProfileName'].lower():
        print(f"Profile Name: {profile['inferenceProfileName']}")
        print(f"Profile ARN: {profile['inferenceProfileArn']}")
        print("---")

Profile Name: US Meta Llama 3.2 11B Instruct
Profile ARN: arn:aws:bedrock:us-west-2:055029294644:inference-profile/us.meta.llama3-2-11b-instruct-v1:0
---
Profile Name: US Meta Llama 3.2 90B Instruct
Profile ARN: arn:aws:bedrock:us-west-2:055029294644:inference-profile/us.meta.llama3-2-90b-instruct-v1:0
---
Profile Name: US Meta Llama 3.2 3B Instruct
Profile ARN: arn:aws:bedrock:us-west-2:055029294644:inference-profile/us.meta.llama3-2-3b-instruct-v1:0
---
Profile Name: US Meta Llama 3.2 1B Instruct
Profile ARN: arn:aws:bedrock:us-west-2:055029294644:inference-profile/us.meta.llama3-2-1b-instruct-v1:0
---
Profile Name: US Meta Llama 3.1 8B Instruct
Profile ARN: arn:aws:bedrock:us-west-2:055029294644:inference-profile/us.meta.llama3-1-8b-instruct-v1:0
---
Profile Name: US Meta Llama 3.1 70B Instruct
Profile ARN: arn:aws:bedrock:us-west-2:055029294644:inference-profile/us.meta.llama3-1-70b-instruct-v1:0
---
Profile Name: US Meta Llama 3.3 70B Instruct
Profile ARN: arn:aws:bedrock:us-west-

In [8]:
bedrock_client = boto3.client(
    service_name="bedrock",
    region_name="us-west-2"
)

bedrock_client.list_foundation_models(byProvider="meta", byCustomizationType="FINE_TUNING")


{'ResponseMetadata': {'RequestId': '99734254-31ad-4871-b044-3eeae12072a4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 14 Jul 2025 17:09:27 GMT',
   'content-type': 'application/json',
   'content-length': '4131',
   'connection': 'keep-alive',
   'x-amzn-requestid': '99734254-31ad-4871-b044-3eeae12072a4'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/meta.llama3-1-8b-instruct-v1:0:128k',
   'modelId': 'meta.llama3-1-8b-instruct-v1:0:128k',
   'modelName': 'Llama 3.1 8B Instruct',
   'providerName': 'Meta',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': ['FINE_TUNING', 'DISTILLATION'],
   'inferenceTypesSupported': ['PROVISIONED'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/meta.llama3-1-70b-instruct-v1:0:128k',
   'modelId': 'meta.llama3-1-70b-instruct-v1:0:128k',
   'mod

In [2]:
from __future__ import annotations

import json
import logging
import os
import random
import time
from pathlib import Path
from typing import List

import boto3
from botocore.exceptions import ClientError

AWS_REGION = os.getenv("AWS_REGION", "ap-southeast-2")
os.environ["AWS_PROFILE"] = "hackathon"
MODEL_ID = os.getenv(
    "BEDROCK_MODEL_ID",
    "arn:aws:bedrock:ap-southeast-2:055029294644:inference-profile/apac.anthropic.claude-sonnet-4-20250514-v1:0",
)
TOTAL_RECORDS    = 1000
BATCH_SIZE       = 10         # number of samples per Bedrock call
OUTPUT_FILE      = Path("jar_coaching_dataset.jsonl")
MAX_RETRIES      = 3
BACKOFF_BASE_SEC = 0.5

SPENDING_JARS    = ["NEC", "PLY"]
ENCOURAGE_JARS   = ["FFA", "EDU", "GIV"]
ALL_JARS         = SPENDING_JARS + ENCOURAGE_JARS


logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[
        logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

bedrock_rt = boto3.client("bedrock-runtime", region_name=AWS_REGION)


def bedrock_with_retry(client, **kwargs) -> dict:
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            return client.converse(**kwargs)
        except ClientError as e:
            if attempt == MAX_RETRIES:
                logger.error("Max retries reached → %s", e)
                raise
            backoff = BACKOFF_BASE_SEC * (2 ** (attempt - 1)) * random.uniform(0.8, 1.2)
            logger.warning("Throttled (%s). Retrying in %.2fs …", e.response["Error"]["Code"], backoff)
            time.sleep(backoff)


# PROMPT TEMPLATE (kept identical to inference-time prompt)
PROMPT_TEMPLATE = """Human: Bạn là một trợ lý tài chính AI vui tính, nhí nhảnh hài hước nhưng vẫn rất thông minh. Nhiệm vụ của bạn là phân tích dữ liệu chi tiêu từ các "lọ" (jars) của người dùng và đưa ra lời khuyên hoặc cảnh báo sắc bén dưới dạng một danh sách JSON.

**QUY TẮC:**
1.  **Phân tích:**
    - Với các lọ chi tiêu thông thường (NEC, PLY), nếu `actual_daily_spending` < `ideal_daily_spending`, đây là vấn đề cần cảnh báo (chi tiêu quá nhanh).
    - Với các lọ khuyến khích (FFA, EDU, GIV), nếu `actual_daily_spending` > `ideal_daily_spending`, đây là vấn đề cần nhắc nhở (chi tiêu quá chậm).
2.  **Định dạng đầu ra:** Phải là một danh sách (array) các đối tượng JSON hợp lệ, không chứa bất kỳ văn bản nào khác ngoài danh sách JSON này.
3.  **Cấu trúc đối tượng JSON:** Mỗi đối tượng phải chứa các key: `jar_code`, `issue`, `recommendation`, `priority`.
    - `jar_code`: Mã của lọ (ví dụ: "PLY").
    - `issue`: Mô tả ngắn gọn vấn đề ("Chi tiêu cao bất thường" hoặc "Chi tiêu thấp bất thường").
    - `recommendation`: Một lời khuyên/cảnh báo ngắn gọn, hài hước, thông minh bằng tiếng Việt.
    - `priority`: Mức độ ưu tiên ("high", "medium", hoặc "low").

**DỮ LIỆU CẦN PHÂN TÍCH:**
<data>
{jar_summary}
</data>

Bây giờ, hãy tạo danh sách JSON dựa trên dữ liệu trên.

Assistant:
```json
"""

# data synthesis functions
def vnd(val: int) -> str:
    return f"{val:,.0f}đ".replace(",", ".")


def random_jar_line(code: str) -> tuple[str, int, int]:
    """Return a jar summary line that **triggers** the rule."""
    while True:
        ideal = random.randint(100_000, 1_000_000)
        actual = int(ideal * random.uniform(0.5, 1.5))

        # enforce trigger condition with 120 % threshold
        if code in SPENDING_JARS and actual < ideal * 1.2:
            break
        if code in ENCOURAGE_JARS and actual > ideal * 1.2:
            break

    line = (
        f"- Lọ {code}: Mức chi tiêu lý tưởng mỗi ngày là {vnd(ideal)}, "
        f"nhưng mức chi tiêu thực tế còn lại mỗi ngày của bạn là {vnd(actual)}."
    )
    return line, ideal, actual


def build_prompt() -> tuple[str, dict]:
    """Generate ONE scenario prompt and structured jar data."""
    n_jars = random.randint(1, len(ALL_JARS))
    picked = random.sample(ALL_JARS, k=n_jars)

    lines, jars_data = [], []
    for code in picked:
        l, ideal, actual = random_jar_line(code)
        lines.append(l)
        jars_data.append(
            {"jar_code": code, "ideal_daily_spending": ideal, "actual_daily_spending": actual}
        )

    prompt_str = PROMPT_TEMPLATE.format(jar_summary="\n".join(lines))
    return prompt_str, jars_data


def main() -> None:
    # OUTPUT_FILE.unlink(missing_ok=True)
    done = 0

    while done < TOTAL_RECORDS:
        batch_size = min(BATCH_SIZE, TOTAL_RECORDS - done)
        scenario_prompts: List[str] = []
        for _ in range(batch_size):
            p, _ = build_prompt()
            scenario_prompts.append(p)

        # Build user prompt asking the model to answer each scenario on its own line
        BATCH_USER_PROMPT = "\n\n".join(
            f"### SCENARIO {idx+1}\n{scenario_prompts[idx]}" for idx in range(batch_size)
        )
        clarification = (
            "Trả về đúng "
            f"{batch_size} kết quả, mỗi kết quả trên **một dòng duy nhất**, "
            "theo đúng thứ tự các SCENARIO, và không kèm bất kỳ ký tự nào khác."
        )

        user_prompt = BATCH_USER_PROMPT + "\n\n" + clarification

        # bedrock call
        resp = bedrock_with_retry(
            bedrock_rt,
            modelId=MODEL_ID,
            system=[{"text": "You are a digital assistant with a friendly personality"}],
            messages=[{"role": "user", "content": [{"text": user_prompt}]}],
        )
        raw = resp["output"]["message"]["content"][0]["text"].strip()

        # align answers with prompts
        answers = [l for l in raw.splitlines() if l.strip()]
        if len(answers) != batch_size:
            logger.warning("Expected %d lines, got %d → retry batch", batch_size, len(answers))
            continue

        # validate and write to file
        with OUTPUT_FILE.open("a", encoding="utf-8") as f:
            for p, c in zip(scenario_prompts, answers):
                # clean accidental code-fences
                if c.startswith("```"):
                    c = c.strip("`").lstrip("json").strip()
                try:
                    json.loads(c)
                except json.JSONDecodeError:
                    logger.warning("Malformed JSON skipped.")
                    continue
                f.write(json.dumps({"prompt": p, "completion": c}, ensure_ascii=False) + "\n")
                done += 1

        logger.info("Progress: %d / %d", done, TOTAL_RECORDS)

    logger.info("Dataset saved → %s", OUTPUT_FILE.resolve())


if __name__ == "__main__":
    main()


2025-07-14 21:35:09 [INFO] Progress: 10 / 1000
2025-07-14 21:35:27 [INFO] Progress: 20 / 1000
2025-07-14 21:35:43 [INFO] Progress: 30 / 1000
2025-07-14 21:36:07 [WARNING] Expected 10 lines, got 90 → retry batch
2025-07-14 21:36:39 [INFO] Progress: 40 / 1000
2025-07-14 21:37:09 [INFO] Progress: 50 / 1000
2025-07-14 21:37:28 [WARNING] Expected 10 lines, got 11 → retry batch
2025-07-14 21:37:34 [WARNING] Throttled (ThrottlingException). Retrying in 0.41s …
2025-07-14 21:37:41 [WARNING] Throttled (ThrottlingException). Retrying in 0.97s …
2025-07-14 21:38:23 [INFO] Progress: 60 / 1000
2025-07-14 21:38:44 [INFO] Progress: 70 / 1000
2025-07-14 21:39:20 [INFO] Progress: 80 / 1000
2025-07-14 21:39:40 [WARNING] Expected 10 lines, got 11 → retry batch
2025-07-14 21:40:07 [WARNING] Expected 10 lines, got 90 → retry batch
2025-07-14 21:40:50 [WARNING] Malformed JSON skipped.
2025-07-14 21:40:50 [INFO] Progress: 89 / 1000
2025-07-14 21:41:19 [INFO] Progress: 99 / 1000
2025-07-14 21:41:57 [WARNING] 

In [ ]:
from __future__ import annotations

import json
import logging
import os
import random
import time
from pathlib import Path
from typing import List

import boto3
from botocore.exceptions import ClientError

AWS_REGION = os.getenv("AWS_REGION", "ap-southeast-2")
os.environ["AWS_PROFILE"] = "hackathon"
MODEL_ID = os.getenv(
    "BEDROCK_MODEL_ID",
    "arn:aws:bedrock:ap-southeast-2:055029294644:inference-profile/apac.anthropic.claude-3-5-sonnet-20241022-v2:0",
)
TOTAL_RECORDS    = 600
BATCH_SIZE       = 10         # number of samples per Bedrock call
OUTPUT_FILE      = Path("jar_coaching_dataset.jsonl")
MAX_RETRIES      = 3
BACKOFF_BASE_SEC = 0.5

SPENDING_JARS    = ["NEC", "PLY"]
ENCOURAGE_JARS   = ["FFA", "EDU", "GIV"]
ALL_JARS         = SPENDING_JARS + ENCOURAGE_JARS


logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[
        logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

bedrock_rt = boto3.client("bedrock-runtime", region_name=AWS_REGION)


def bedrock_with_retry(client, **kwargs) -> dict:
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            return client.converse(**kwargs)
        except ClientError as e:
            if attempt == MAX_RETRIES:
                logger.error("Max retries reached → %s", e)
                raise
            backoff = BACKOFF_BASE_SEC * (2 ** (attempt - 1)) * random.uniform(0.8, 1.2)
            logger.warning("Throttled (%s). Retrying in %.2fs …", e.response["Error"]["Code"], backoff)
            time.sleep(backoff)


# PROMPT TEMPLATE (kept identical to inference-time prompt)
PROMPT_TEMPLATE = """Human: Bạn là một trợ lý tài chính AI vui tính, nhí nhảnh hài hước nhưng vẫn rất thông minh. Nhiệm vụ của bạn là phân tích dữ liệu chi tiêu từ các "lọ" (jars) của người dùng và đưa ra lời khuyên hoặc cảnh báo sắc bén dưới dạng một danh sách JSON.

**QUY TẮC:**
1.  **Phân tích:**
    - Với các lọ chi tiêu thông thường (NEC, PLY), nếu `actual_daily_spending` < `ideal_daily_spending`, đây là vấn đề cần cảnh báo (chi tiêu quá nhanh).
    - Với các lọ khuyến khích (FFA, EDU, GIV), nếu `actual_daily_spending` > `ideal_daily_spending`, đây là vấn đề cần nhắc nhở (chi tiêu quá chậm).
2.  **Định dạng đầu ra:** Phải là một danh sách (array) các đối tượng JSON hợp lệ, không chứa bất kỳ văn bản nào khác ngoài danh sách JSON này.
3.  **Cấu trúc đối tượng JSON:** Mỗi đối tượng phải chứa các key: `jar_code`, `issue`, `recommendation`, `priority`.
    - `jar_code`: Mã của lọ (ví dụ: "PLY").
    - `issue`: Mô tả ngắn gọn vấn đề ("Chi tiêu cao bất thường" hoặc "Chi tiêu thấp bất thường").
    - `recommendation`: Một lời khuyên/cảnh báo ngắn gọn, hài hước, thông minh bằng tiếng Việt.
    - `priority`: Mức độ ưu tiên ("high", "medium", hoặc "low").

**DỮ LIỆU CẦN PHÂN TÍCH:**
<data>
{jar_summary}
</data>

Bây giờ, hãy tạo danh sách JSON dựa trên dữ liệu trên.

Assistant:
```json
"""

# data synthesis functions
def vnd(val: int) -> str:
    return f"{val:,.0f}đ".replace(",", ".")


def random_jar_line(code: str) -> tuple[str, int, int]:
    """Return a jar summary line that **triggers** the rule."""
    while True:
        ideal = random.randint(100_000, 1_000_000)
        actual = int(ideal * random.uniform(0.5, 1.5))

        # enforce trigger condition with 120 % threshold
        if code in SPENDING_JARS and actual < ideal * 1.2:
            break
        if code in ENCOURAGE_JARS and actual > ideal * 1.2:
            break

    line = (
        f"- Lọ {code}: Mức chi tiêu lý tưởng mỗi ngày là {vnd(ideal)}, "
        f"nhưng mức chi tiêu thực tế còn lại mỗi ngày của bạn là {vnd(actual)}."
    )
    return line, ideal, actual


def build_prompt() -> tuple[str, dict]:
    """Generate ONE scenario prompt and structured jar data."""
    n_jars = random.randint(1, len(ALL_JARS))
    picked = random.sample(ALL_JARS, k=n_jars)

    lines, jars_data = [], []
    for code in picked:
        l, ideal, actual = random_jar_line(code)
        lines.append(l)
        jars_data.append(
            {"jar_code": code, "ideal_daily_spending": ideal, "actual_daily_spending": actual}
        )

    prompt_str = PROMPT_TEMPLATE.format(jar_summary="\n".join(lines))
    return prompt_str, jars_data


def main() -> None:
    # OUTPUT_FILE.unlink(missing_ok=True)
    done = 0

    while done < TOTAL_RECORDS:
        batch_size = min(BATCH_SIZE, TOTAL_RECORDS - done)
        scenario_prompts: List[str] = []
        for _ in range(batch_size):
            p, _ = build_prompt()
            scenario_prompts.append(p)

        # Build user prompt asking the model to answer each scenario on its own line
        BATCH_USER_PROMPT = "\n\n".join(
            f"### SCENARIO {idx+1}\n{scenario_prompts[idx]}" for idx in range(batch_size)
        )
        clarification = (
            "Trả về đúng "
            f"{batch_size} kết quả, mỗi kết quả trên **một dòng duy nhất**, "
            "theo đúng thứ tự các SCENARIO, và không kèm bất kỳ ký tự nào khác."
        )

        user_prompt = BATCH_USER_PROMPT + "\n\n" + clarification

        # bedrock call
        resp = bedrock_with_retry(
            bedrock_rt,
            modelId=MODEL_ID,
            system=[{"text": "You are a digital assistant with a friendly personality"}],
            messages=[{"role": "user", "content": [{"text": user_prompt}]}],
        )
        raw = resp["output"]["message"]["content"][0]["text"].strip()

        # align answers with prompts
        answers = [l for l in raw.splitlines() if l.strip()]
        if len(answers) != batch_size:
            logger.warning("Expected %d lines, got %d → retry batch", batch_size, len(answers))
            continue

        # validate and write to file
        with OUTPUT_FILE.open("a", encoding="utf-8") as f:
            for p, c in zip(scenario_prompts, answers):
                # clean accidental code-fences
                if c.startswith("```"):
                    c = c.strip("`").lstrip("json").strip()
                try:
                    json.loads(c)
                except json.JSONDecodeError:
                    logger.warning("Malformed JSON skipped.")
                    continue
                f.write(json.dumps({"prompt": p, "completion": c}, ensure_ascii=False) + "\n")
                done += 1

        logger.info("Progress: %d / %d", done, TOTAL_RECORDS)

    logger.info("Dataset saved → %s", OUTPUT_FILE.resolve())


if __name__ == "__main__":
    main()


2025-07-15 00:35:50 [INFO] Loading cached SSO token for test_sess
2025-07-15 00:36:17 [INFO] Progress: 10 / 600
2025-07-15 00:36:52 [INFO] Progress: 20 / 600
2025-07-15 00:37:31 [INFO] Progress: 30 / 600
2025-07-15 00:38:09 [INFO] Progress: 40 / 600
2025-07-15 00:38:25 [INFO] Progress: 50 / 600
2025-07-15 00:39:00 [INFO] Progress: 60 / 600
2025-07-15 00:39:27 [INFO] Progress: 70 / 600
2025-07-15 00:40:03 [INFO] Progress: 80 / 600
2025-07-15 00:40:43 [INFO] Progress: 90 / 600
2025-07-15 00:41:15 [INFO] Progress: 100 / 600
